In [8]:
import csv
import cv2
import numpy as np

lines = []
with open("driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
counter = 0
if(False):
    for line in lines:
        for i in range(3):
            sourcepath = line[i]
            filename = sourcepath.split('\\')[-1]
            currentpath = "IMG/" + filename
            image = cv2.imread(currentpath)
            images.append(image)
            measurements.append(float(line[3]))
            counter = counter + 1
        
for row in lines:
    steering_center = float(row[3])

    # create adjusted steering measurements for the side camera images
    correction = 0.2 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction

    # read in images from center, left and right cameras
    path = "..." # fill in the path to your training IMG directory
    img_center = cv2.imread("IMG/" + row[0].split('\\')[-1])
    img_left = cv2.imread("IMG/" + row[1].split('\\')[-1])
    img_right = cv2.imread("IMG/" + row[2].split('\\')[-1])

    # add images and angles to data set
    images.extend([img_center, img_left, img_right])
    measurements.extend([steering_center, steering_left, steering_right])
    
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement*-1.0)
    
x_train = np.array(images)
y_train = np.array(measurements)


from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: (x/255.0), input_shape = (160, 320, 3)))
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))

model.add(Conv2D(16, (5, 5), strides=(2, 2), activation="relu"))
model.add(Dropout(0.3))
#model.add(MaxPooling2D())
model.add(Conv2D(32, (5, 5), strides=(2, 2), activation="relu"))
model.add(Dropout(0.3))
model.add(MaxPooling2D())
model.add(Conv2D(64, (5, 5), strides=(1, 1), activation="relu"))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(x_train, y_train, validation_split=0.2, shuffle = True, epochs = 15)
model.save('model.h5')

scores = model.evaluate(x_train, y_train)
print(scores)

Train on 4646 samples, validate on 1162 samples
Epoch 1/15
4646/4646 [==============================] - 9s - loss: 0.5158 - val_loss: 0.0331
Epoch 2/15
4646/4646 [==============================] - 9s - loss: 0.0384 - val_loss: 0.0323
Epoch 3/15
4646/4646 [==============================] - 9s - loss: 0.0370 - val_loss: 0.0339
Epoch 4/15
4646/4646 [==============================] - 9s - loss: 0.0358 - val_loss: 0.0341
Epoch 5/15
4646/4646 [==============================] - 9s - loss: 0.0335 - val_loss: 0.0317
Epoch 6/15
4646/4646 [==============================] - 9s - loss: 0.0319 - val_loss: 0.0351
Epoch 7/15
4646/4646 [==============================] - 9s - loss: 0.0298 - val_loss: 0.0324
Epoch 8/15
4646/4646 [==============================] - 9s - loss: 0.0290 - val_loss: 0.0321
Epoch 9/15
4646/4646 [==============================] - 9s - loss: 0.0279 - val_loss: 0.0432
Epoch 10/15
4646/4646 [==============================] - 9s - loss: 0.0276 - val_loss: 0.0407
Epoch 11/15
4646/4646